In [7]:
import sys
import json
import pickle
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas, pd_writer
import getpass as gt
import pandas as pd

from snowflake_conn import *

conn = get_connection()

In [9]:
sql = '''SELECT * FROM SANDBOX_CLAIMS.CMS_HCC.PATIENT_RISK_SCORES;'''
risk = read_sql(sql)

print(risk.shape)
risk.head()

,patient_id,raw_risk_score,normalized_risk_score,payment_risk_score,model_version,payment_year,last_update
0,10022,1.014,0.907,0.853,CMS-HCC-V24,2022,2023-08-22 10:57:54.756790-04:00
1,11512,0.600,0.537,0.505,CMS-HCC-V24,2022,2023-08-22 10:57:54.756790-04:00
2,11286,0.786,0.703,0.662,CMS-HCC-V24,2022,2023-08-22 10:57:54.756790-04:00
3,11423,1.014,0.907,0.853,CMS-HCC-V24,2022,2023-08-22 10:57:54.756790-04:00
4,10186,0.000,0.000,0.000,CMS-HCC-V24,2022,2023-08-22 10:57:54.756790-04:00
...,...,...,...,...,...,...,...
995,11116,1.014,0.907,0.853,CMS-HCC-V24,2022,2023-08-22 10:57:54.756790-04:00
996,13088,0.860,0.769,0.724,CMS-HCC-V24,2022,2023-08-22 10:57:54.756790-04:00
997,11649,1.293,1.157,1.089,CMS-HCC-V24,2022,2023-08-22 10:57:54.756790-04:00
998,12923,1.060,0.948,0.892,CMS-HCC-V24,2022,2023-08-22 10:57:54.756790-04:00


In [11]:
sql = '''SELECT * FROM SANDBOX_CLAIMS.PMPM.PMPM_PREP;'''
pmpm = read_sql(sql)

print(pmpm.shape)
pmpm.head()

(31184, 53)


,patient_id,year_month,inpatient_paid,outpatient_paid,office_visit_paid,ancillary_paid,pharmacy_paid,other_paid,acute_inpatient_paid,ambulance_paid,...,outpatient_hospital_or_clinic_allowed,outpatient_psychiatric_allowed,outpatient_rehabilitation_allowed,skilled_nursing_allowed,urgent_care_allowed,total_paid,medical_paid,total_allowed,medical_allowed,last_update
0,10,201604,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2023-08-31 22:58:40.087327-04:00
1,10,201605,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2023-08-31 22:58:40.087327-04:00
2,10,201611,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2023-08-31 22:58:40.087327-04:00
3,10,201612,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2023-08-31 22:58:40.087327-04:00
4,10,201705,0.00,331.81,210.15,143.06,0.00,0.00,0.00,0.00,...,363.18,0.00,0.00,0.00,0.00,685.02,685.02,958.26,958.26,2023-08-31 22:58:40.087327-04:00


In [16]:
# this currently is summing over multiple years
grouped = pmpm.groupby(by="patient_id")[pmpm.columns[2:8]].sum().reset_index(drop=False)
grouped.head()

,patient_id,inpatient_paid,outpatient_paid,office_visit_paid,ancillary_paid,pharmacy_paid,other_paid
0,10,0.00,2461.04,1370.59,413.06,0.00,0.00
1,10000,0.00,3905.39,5374.92,1172.81,0.00,0.00
2,10002,0.00,2125.00,3659.85,547.93,0.00,122.28
3,10004,8820.16,1811.73,8020.40,2228.77,0.00,0.00
4,10007,3052.89,13265.48,5764.27,1169.37,0.00,5626.82


In [17]:
df = risk.merge(grouped, on='patient_id', how="outer", indicator=True)
print(df.shape)
df.head()

(1000, 14)


,patient_id,raw_risk_score,normalized_risk_score,payment_risk_score,model_version,payment_year,last_update,inpatient_paid,outpatient_paid,office_visit_paid,ancillary_paid,pharmacy_paid,other_paid,_merge
0,10022,1.014,0.907,0.853,CMS-HCC-V24,2022,2023-08-22 10:57:54.756790-04:00,0.00,834.78,2475.93,1436.48,0.00,0.00,both
1,11512,0.600,0.537,0.505,CMS-HCC-V24,2022,2023-08-22 10:57:54.756790-04:00,8.34,0.00,37781.09,55.58,0.00,0.00,both
2,11286,0.786,0.703,0.662,CMS-HCC-V24,2022,2023-08-22 10:57:54.756790-04:00,0.00,2439.12,881.28,84.79,0.00,0.00,both
3,11423,1.014,0.907,0.853,CMS-HCC-V24,2022,2023-08-22 10:57:54.756790-04:00,1847.31,19513.47,4741.40,412.65,0.00,6142.75,both
4,10186,0.000,0.000,0.000,CMS-HCC-V24,2022,2023-08-22 10:57:54.756790-04:00,0.00,3850.37,5447.42,1227.67,0.00,0.00,both


In [18]:
df._merge.value_counts()

_merge
both          1000
left_only        0
right_only       0
Name: count, dtype: int64

Summary stats

In [ ]:
close_connection()